In [1]:
from os.path import join
import bnr.azrael as azrael

In [2]:
data_folder = "data"
az2a = azrael.Azrael2analysis()
az2a.create_az(path_az=join(data_folder, "bnr_d001_20231218.csv.gz"))

az2a.split_path()
az2a.get_extension_mimetype()
az2a.az.columns
az = az2a.az
az['nb'] = 1
az = az[az['path0'].isin(['BNR_VERIF', 'BNR_TAMPON', 'BNR_SAUV'])]
az.columns

Index(['name', 'path', 'md5', 'size', 'last_content_modification_date',
       'last_metadata_modification_date', 'path0', 'path1', 'path2', 'path3',
       'path4', 'extension', 'mimetype', 'guessed_extension', 'nb'],
      dtype='object')

In [23]:
az_by_path0 = az.groupby(['path0'])['name'].count().reset_index()
az_by_path0 = az_by_path0.rename(columns={'name':'nb'})
az_by_path0['%'] = (round((az_by_path0['nb'] / az_by_path0['nb'].sum()) * 100, 0)).astype(int)
az_by_path0

,path0,nb,%
0,BNR_SAUV,512461,22
1,BNR_TAMPON,3389,0
2,BNR_VERIF,1785153,78


In [24]:
az_by_path1 = az.groupby(['path1'])['name'].count().reset_index()
az_by_path1 = az_by_path1.rename(columns={'name':'nb'})
az_by_path1['%'] = (round((az_by_path1['nb'] / az_by_path1['nb'].sum()) * 100, 0)).astype(int)
az_by_path1

,path1,nb,%
0,AMR,792236,34
1,ARA,176,0
2,CSV,243,0
3,LAI,1666,0
4,LAR,3827,0
5,MAGASINES,9487,0
6,MDF,19,0
7,MED,1473739,64
8,MUS,2340,0
9,OBS,1802,0


In [26]:
az_by_path2 = az.groupby(['path2'])['name'].count().reset_index()
az_by_path2 = az_by_path2.rename(columns={'name':'nb'})
az_by_path2['%'] = (round((az_by_path2['nb'] / az_by_path2['nb'].sum()) * 100, 0)).astype(int)
az_by_path2.sort_values(by='nb', ascending=False)

,path2,nb,%
70,MED_PRA,1342180,58
32,AMR_EC,373096,16
31,AMR_DEL,122234,5
39,AMR_POP,86848,4
22,AMR_6H,77144,3
...,...,...,...
33,AMR_MIRES,8,0
24,AMR_8FI,5,0
103,non patrimoine,5,0
11,AMR_381W,4,0


In [4]:
az_by_mimetype = az.groupby(['mimetype'])['name'].count().reset_index()
az_by_mimetype = az_by_mimetype.rename(columns={'name':'nb'})
az_by_mimetype['%'] = (round((az_by_mimetype['nb'] / az_by_mimetype['nb'].sum()) * 100, 0)).astype(int)
az_by_mimetype.sort_values(by='nb', ascending=False)

,mimetype,nb,%
12,image/jpeg,609032,27
14,image/tiff,591680,26
7,application/xml,414639,18
1,application/pdf,337205,15
18,text/plain,334134,15
10,audio/x-wav,8282,0
9,audio/mpeg,630,0
20,video/quicktime,124,0
16,model/vnd.mts,38,0
15,image/x-xcf,38,0


In [3]:
gr = az.groupby(['path0', 'path1', 'path2', 'mimetype']).agg({'nb': 'sum', 'size': 'sum'}).reset_index()

In [4]:
gr['size_go'] = gr['size'].apply(lambda x: azrael.convert_size(x, from_size='o', to_size='go'))
gr['size_to'] = gr['size'].apply(lambda x: azrael.convert_size(x, from_size='o', to_size='to'))

In [5]:
gr.to_excel("results/synthese.xlsx", index=False)